In [ ]:
###

In [1]:
from models.latte_t2v import LatteT2V  # 导入模型
from diffusers.models import AutoencoderKL, AutoencoderKLTemporalDecoder
from diffusers.schedulers import PNDMScheduler
import torch
from einops import repeat
import copy

import torch

pretrain_model_path = '/data2/wangtuo/workspace/model/Latte/hf_hub/models--maxin-cn--Latte/t2v_required_models'
model_name = '/data2/wangtuo/workspace/model/Latte/hf_hub/models--maxin-cn--Latte/t2v.pt'
device = "cuda:1"

# T2v
lattet2v_model = LatteT2V.from_pretrained_2d(pretrain_model_path, subfolder="transformer", video_length = 16).to(device) 
from transformers import T5EncoderModel, T5Tokenizer
checkpoint = torch.load(model_name, map_location=lambda storage, loc: storage)
lattet2v_model.load_state_dict(checkpoint['model'])
# 把pipeline加载明白

from sample.pipeline_videogen import VideoGenPipeline


vae = AutoencoderKL.from_pretrained(pretrain_model_path, subfolder="vae", torch_dtype=torch.float16).to(device)
vae.half()
print(vae.dtype)
tokenizer = T5Tokenizer.from_pretrained(pretrain_model_path, subfolder="tokenizer")
text_encoder = T5EncoderModel.from_pretrained(pretrain_model_path, subfolder="text_encoder", torch_dtype=torch.float16).to(device)
scheduler = PNDMScheduler.from_pretrained(pretrain_model_path, 
                                                  subfolder="scheduler",
                                                  beta_start=0.0001, 
                                                  beta_end=0.02, 
                                                  beta_schedule="linear",
                                                variance_type="learned_range")
vae.eval()
text_encoder.eval()

videogen_pipeline = VideoGenPipeline(vae=vae, 
                                text_encoder=text_encoder, 
                                tokenizer=tokenizer, 
                                scheduler=scheduler, 
                                transformer=lattet2v_model).to(device)

/data2/wangtuo/Miniconda/envs/latte2/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


torch.float16


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
prompt = ['Rihanna']
encoder_hidden_state, encoder_hidden_state_mask = videogen_pipeline.get_sentence_embedding  (prompt, 
                        video_length=16, 
                        height=512, 
                        width=512, 
                        num_inference_steps=100,
                        guidance_scale=9.5,
                        enable_temporal_attentions=True,
                        num_images_per_prompt=1,
                        mask_feature=True,
                        enable_vae_temporal_decoder=False
                        )
prompt = ['Beyoncé']
encoder_hidden_state2, encoder_hidden_state_mask2 = videogen_pipeline.get_sentence_embedding(prompt, 
                        video_length=16, 
                        height=512, 
                        width=512, 
                        num_inference_steps=100,
                        guidance_scale=9.5,
                        enable_temporal_attentions=True,
                        num_images_per_prompt=1,
                        mask_feature=True,
                        enable_vae_temporal_decoder=False
                        )
print(encoder_hidden_state.shape)
print(encoder_hidden_state2.shape)

torch.Size([32, 6, 1152])
torch.Size([32, 6, 1152])


In [3]:
print(encoder_hidden_state[16,:2,:10], encoder_hidden_state[17,:2,:10])
atol=1e-8
# 判断是否相等
print(torch.allclose(encoder_hidden_state[16,0,512], encoder_hidden_state2[16,0,512], atol=1e-8))
# # 计算两个张量的差异
# difference = torch.abs(encoder_hidden_state - encoder_hidden_state2)

# # 找到超过允许误差范围的位置
# different_indices = torch.nonzero(difference > atol)

# # 如果存在不同的值，打印不同的位置及其值
# if different_indices.numel() > 0:
#     print(f"在误差范围 {atol} 内，不同的位置及其值如下：")
#     for idx in different_indices:
#         idx = tuple(idx.tolist())
#         print(f"位置 {idx} - encoder_hidden_state: {encoder_hidden_state[idx]}, encoder_hidden_state2: {encoder_hidden_state2[idx]}")
# else:
#     print("所有值在允许误差范围内相同。")

tensor([[-0.0250, -0.2516,  0.1268,  0.0687,  0.1209, -0.1931, -0.2395,  0.1474,
          0.2589,  0.0891],
        [ 0.2702, -0.0277, -0.2992, -0.1014,  0.3698, -0.0843,  0.1775, -0.0253,
         -0.1433,  0.1125]], device='cuda:1') tensor([[-0.0250, -0.2516,  0.1268,  0.0687,  0.1209, -0.1931, -0.2395,  0.1474,
          0.2589,  0.0891],
        [ 0.2702, -0.0277, -0.2992, -0.1014,  0.3698, -0.0843,  0.1775, -0.0253,
         -0.1433,  0.1125]], device='cuda:1')
False


In [4]:
# 一些参数配置
erase_scale = 0.1
preserve_scale =0.1
is_use_k = True
old_texts_ = ['Biden is running']
new_texts_ = ['Messi is running']
ret_texts = ['A.J.Casson']

In [5]:
def get_prompt_embedding(videogen_pipeline, prompt):
    encoder_hidden_state, encoder_hidden_state_mask = videogen_pipeline.get_sentence_embedding(prompt, 
                        video_length=16, 
                        height=512, 
                        width=512, 
                        num_inference_steps=4,
                        guidance_scale=7.5,
                        enable_temporal_attentions=True,
                        num_images_per_prompt=1,
                        mask_feature=True,
                        enable_vae_temporal_decoder=False
                        )
    return encoder_hidden_state[16,:,:]

In [13]:
find_model = videogen_pipeline.transformer.named_modules()
print(videogen_pipeline.transformer)
for name, module in videogen_pipeline.transformer.named_modules():
    # 27个
    if (
        isinstance(module, torch.nn.Linear)
        and "ff.net" in name
        and not "proj" in name
        and not "temporal" in name
    ):
        print(name)
        print(module)
        print("------------------")

LatteT2V(
  (pos_embed): PatchEmbed(
    (proj): Conv2d(4, 1152, kernel_size=(2, 2), stride=(2, 2))
  )
  (transformer_blocks): ModuleList(
    (0-27): 28 x BasicTransformerBlock(
      (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=False)
      (attn1): Attention(
        (to_q): Linear(in_features=1152, out_features=1152, bias=True)
        (to_k): Linear(in_features=1152, out_features=1152, bias=True)
        (to_v): Linear(in_features=1152, out_features=1152, bias=True)
        (to_out): ModuleList(
          (0): Linear(in_features=1152, out_features=1152, bias=True)
          (1): Dropout(p=0.0, inplace=False)
        )
      )
      (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=False)
      (attn2): Attention(
        (to_q): Linear(in_features=1152, out_features=1152, bias=True)
        (to_k): Linear(in_features=1152, out_features=1152, bias=True)
        (to_v): Linear(in_features=1152, out_features=1152, bias=True)
        (to_out): ModuleList(
       

In [6]:
transformers_block = lattet2v_model.transformer_blocks.named_children()
ca_layers = []
print(f'寻找交叉注意力')
for transformer_block in transformers_block: 
    # 对于每一层transformer_block,选择basic_block
    basic_transformer_block = transformer_block[1]
    basic_transformer_layers = basic_transformer_block.named_children()
    # 遍历，找到基本层
    for layer in basic_transformer_layers:
        if 'attn2' in layer[0]:
            ca_layers.append(layer[1])
            
print(ca_layers[0].to_v.weight)  # 28个交叉注意力层

# 这里的encoder_hidden_states还是作为k,v的输入
projection_matrices = [l.to_v for l in ca_layers]
og_matrices = [copy.deepcopy(l.to_v) for l in ca_layers]
if is_use_k:
    projection_matrices = projection_matrices + [l.to_k for l in ca_layers]
    og_matrices = og_matrices + [copy.deepcopy(l.to_k) for l in ca_layers]
print(projection_matrices)

# 这里面很重要的一部分，ca_layers就是attention
# 重置参数,这部分可以修改一下
num_caption_layer = len(ca_layers)
for idx, layer in enumerate(ca_layers):
    layer.to_v = copy.deepcopy(projection_matrices[idx])
    projection_matrices[idx] = layer.to_v
    if is_use_k:
        layer.to_k = copy.deepcopy(og_matrices[num_caption_layer+idx])
        projection_matrices[num_caption_layer+idx] = layer.to_k
        
old_texts = []
new_texts = []
for old_text, new_text in zip(old_texts_, new_texts_):
    old_texts.append(old_text)
    n_t = new_text
    if n_t == '':
        n_t = ' '
    new_texts.append(n_t)

lamb = 1

寻找交叉注意力
Parameter containing:
tensor([[ 0.0270, -0.0089,  0.0713,  ..., -0.0623, -0.0364,  0.0031],
        [-0.0012,  0.0181,  0.0084,  ..., -0.0404,  0.0020,  0.0161],
        [ 0.0142, -0.0662, -0.0083,  ..., -0.0006,  0.0634, -0.0402],
        ...,
        [-0.0273, -0.0215, -0.0196,  ..., -0.0344,  0.0239,  0.0042],
        [ 0.0031, -0.0113, -0.0506,  ..., -0.0469,  0.0275, -0.0688],
        [-0.0072,  0.0164,  0.0131,  ...,  0.0002,  0.0067,  0.0363]],
       device='cuda:1', requires_grad=True)
[LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), LoRACompatibleLinear(in_features=1152, out_features=1152, bias=True), L

In [7]:
# # 重新写一个概念映射
# for layer_num in range(len(projection_matrices)):
#     with torch.no_grad():
#         position1 = torch.zeros((1152, 1152)).to(device)
#         position2 = torch.zeros((1152, 1152)).to(device)
#         position3 = torch.zeros((1152, 1152)).to(device)
#         position4 = torch.zeros((1152, 1152)).to(device)

#         for old_text, new_text in zip(old_texts, new_texts):
#             # 获取missi c^T
#             new_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, new_text)  # [3,1152]
#             # vi
#             old_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, old_text)
#             layer = projection_matrices[layer_num]
#             vi = layer(new_embeds_cap_spatial).detach()  # [3,1152]
            
#             position1 += old_embeds_cap_spatial.transpose(0, 1) @ vi
            
            
#             position3 += old_embeds_cap_spatial.transpose(0, 1) @ old_embeds_cap_spatial
        
#         for remain_text in ret_texts:
#             remain_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, remain_text)
#             layer = projection_matrices[layer_num]
#             vi = layer(remain_embeds_cap_spatial).detach()
#             position2 += remain_embeds_cap_spatial.transpose(0, 1) @ vi
#             position4 += remain_embeds_cap_spatial.transpose(0, 1) @ remain_embeds_cap_spatial
#         mat1 = position1 + position2
#         mat2 = position3 + position4
        
#         projection_matrices[layer_num].weight = torch.nn.Parameter(mat1 @ torch.inverse(mat2))
#         if layer_num == 0:
#             print(mat1)
#             print(mat2)
#             print(projection_matrices[layer_num].weight)
        

In [8]:
# import numpy as np


# ans = mat2.cpu().numpy()
# ans
# def is_full_rank_matrix_C(A):
#     # 计算矩阵的行阶梯形式
#     rref = np.linalg.matrix_rank(A)
#     return rref == A.shape[0]

# print(is_full_rank_matrix_C(ans))

In [9]:
# 下一步的修改计划：使用直接对齐

In [10]:
for layer_num in range(len(projection_matrices)):
    with torch.no_grad():
        # 不改变梯度, 这是后面推导的公式
        mat1 = lamb * projection_matrices[layer_num].weight
        mat2 = lamb * torch.eye(projection_matrices[layer_num].weight.shape[1], device = projection_matrices[layer_num].weight.device)
        for idx, t in enumerate(zip(old_texts, new_texts)):
            print(f't{t}')
            old_text, new_text = t
            old_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, old_text)
            new_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, new_text)
            print(f'old_embeds_cap_spatial{old_embeds_cap_spatial.shape},new_embeds_cap_spatial{new_embeds_cap_spatial.shape}')
            context = old_embeds_cap_spatial.detach()
            # print(f'context{context.shape}')
            values = []
            
            with torch.no_grad():
                # for layer in projection_matrices:
                #     values.append(layer(new_embeds_cap_spatial.detach()))
                for layer in projection_matrices:
                    o_embs = layer(old_embeds_cap_spatial).detach()  # v*
                    u = o_embs
                    u = u / u.norm()
                    
                    new_embs = layer(new_embeds_cap_spatial).detach()
                    new_embs_proj = (u*new_embs).sum()
                    
                    traget = new_embs - (new_embs_proj * u)
                    values.append(traget.detach())
                    
            
            context_vector = context # c [token,1152]
            context_vector_T = context.transpose(0, 1)  # [1152, token]
            
            value_vector = values[layer_num]
            for_mat1 = (context_vector_T @ value_vector)  # [1152,1152]
            for_mat2 = (context_vector_T @ context_vector)  # [1152,1152]
            # print(f'format1{for_mat1.shape},format2{for_mat2.shape},mat1{mat1.shape}')
            mat1 += erase_scale*for_mat1
            mat2 += erase_scale*for_mat2
            
        # # 对于其他要保存的参数，我们要保存下来
        # for old_text, new_text in zip(ret_texts, ret_texts):
        #     print(f'old_text{old_text},new_text{new_text}')
        #     old_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, old_text)
        #     new_embeds_cap_spatial = get_prompt_embedding(videogen_pipeline, new_text)
            
            
        #     context = old_embeds_cap_spatial.detach()
        #     values = []
            
        #     with torch.no_grad():
        #         for layer in projection_matrices:
        #             values.append(layer(new_embeds_cap_spatial.detach()))
        #     context_vector = context # c [32,120,1152]
        #     context_vector_T = context.transpose(0, 1)  # cT  [32,1152,120] []
            
        #     value_vector = values[layer_num]
        #     for_mat1 = (context_vector_T @ value_vector)  # [1152,1152]
        #     for_mat2 = (context_vector_T @ context_vector)
        #     mat1 += preserve_scale*for_mat1
        #     mat2 += preserve_scale*for_mat2
        
        projection_matrices[layer_num].weight = torch.nn.Parameter((mat1 @ torch.inverse(mat2)))
        
checkpoint = { "model": lattet2v_model.state_dict() }
torch.save(checkpoint, 'lattet2v_model2.pt')

t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_embeds_cap_spatialtorch.Size([5, 1152])
t('Biden is running', 'Messi is running')
old_embeds_cap_spatialtorch.Size([5, 1152]),new_

In [11]:
# 这里面我要确认是否真的改变参数
l_to_v_old = ca_layers[0].to_v.weight
print(l_to_v_old)
l_to_v_new = projection_matrices[0].weight
print(l_to_v_new)
og_to_v = og_matrices[0].weight
print(og_to_v)

Parameter containing:
tensor([[ 0.0272, -0.0230,  0.0790,  ..., -0.0552, -0.0358,  0.0142],
        [-0.0074,  0.0244, -0.0025,  ..., -0.0375, -0.0101,  0.0196],
        [ 0.0248, -0.0945,  0.0197,  ...,  0.0168,  0.0602, -0.0135],
        ...,
        [-0.0385,  0.0033, -0.0467,  ..., -0.0477,  0.0311, -0.0164],
        [ 0.0092, -0.0142, -0.0447,  ..., -0.0395,  0.0160, -0.0565],
        [-0.0242,  0.0240, -0.0378,  ..., -0.0009, -0.0251,  0.0126]],
       device='cuda:1', requires_grad=True)
Parameter containing:
tensor([[ 0.0272, -0.0230,  0.0790,  ..., -0.0552, -0.0358,  0.0142],
        [-0.0074,  0.0244, -0.0025,  ..., -0.0375, -0.0101,  0.0196],
        [ 0.0248, -0.0945,  0.0197,  ...,  0.0168,  0.0602, -0.0135],
        ...,
        [-0.0385,  0.0033, -0.0467,  ..., -0.0477,  0.0311, -0.0164],
        [ 0.0092, -0.0142, -0.0447,  ..., -0.0395,  0.0160, -0.0565],
        [-0.0242,  0.0240, -0.0378,  ..., -0.0009, -0.0251,  0.0126]],
       device='cuda:1', requires_grad=True)
